#### Persiapan

In [1]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedKFold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load data
df = pd.read_csv('heart_statlog_cleveland_hungary_final_cleaned.csv')
display(df)
X_data = df.copy().drop(columns=['target'])
y_data = df.copy()['target']

FileNotFoundError: [Errno 2] No such file or directory: 'heart_statlog_cleveland_hungary_final_cleaned.csv'

#### Model Logistic Regression
Alasan Utama: Sederhana, cepat, dan efektif untuk klasifikasi biner.

Penjelasan: Logistic Regression adalah model yang sangat umum untuk masalah klasifikasi biner, seperti memprediksi apakah seseorang memiliki penyakit jantung (target = 1) atau tidak (target = 0). Karena data yang Anda miliki mungkin tidak terlalu kompleks atau sangat besar, Logistic Regression dapat memberi hasil yang cukup baik dalam waktu yang singkat.

Kenapa Memilih Ini: Ini adalah model baseline yang cepat untuk melihat seberapa baik data bisa diprediksi dengan model sederhana. Selain itu, model ini memberikan interpretabilitas yang tinggi, yang berarti kita bisa mengetahui faktor-faktor yang mempengaruhi keputusan model (misalnya, apakah usia atau tekanan darah mempengaruhi risiko penyakit jantung).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform
from scipy.stats import loguniform, randint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


X = X_data.copy().to_numpy()
y = y_data.copy().to_numpy()


# Function untuk membuat pipeline model: Pipeline menyusun preprocessing (StandardScaler) dan model LogisticRegression jadi satu.
def create_pipeline(C=1.0):
    return Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(C=C, max_iter=1000, solver='liblinear'))
    ])
# C: regularisasi (semakin kecil = regulasi makin kuat).
# Solver liblinear cocok untuk dataset kecil.

param_dist = {
    'clf__C': loguniform(0.001, 100),
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['saga'],
    # 'clf__l1_ratio': [0.1, 0.5, 0.9],
    'clf__max_iter': randint(2000, 5000)
}

# Hyperparameter settings
param_grid = {
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['saga'],
    # 'clf__l1_ratio': [0.1, 0.5, 0.9],
    'clf__max_iter': [5000, 10000]
}

# Confusion matrix memberikan visualisasi tentang performa model dalam memprediksi kelas positif dan negatif, serta berapa banyak kesalahan yang terjadi (false positives dan false negatives).
def plot_multiple_confusion_matrices(conf_matrices, title):
    n = len(conf_matrices)
    fig, axes = plt.subplots(1, n, figsize=(3.5 * n, 3.5))

    if n == 1:
        axes = [axes]

    for i, (cm, name) in enumerate(conf_matrices):
        ConfusionMatrixDisplay(confusion_matrix=cm).plot(ax=axes[i], cmap='Blues', colorbar=False)
        axes[i].set_title(name)

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()  # Pastikan hanya panggil show sekali di sini
    plt.close(fig)  # ⬅⬅⬅ Tambahkan ini untuk menghindari dobel
    
# Fungsi evaluasi untuk KFold dan StratifiedKFold
def evaluate_with_cv(cv_method, use_grid=False, use_random=False, show_conf_matrix=False):
    default_scores = []
    grid_scores = []
    random_scores = []
    fold_num = 1

    # Untuk menyimpan semua confusion matrices
    conf_matrices_default = []
    conf_matrices_grid = []
    conf_matrices_random = []

    for train_idx, test_idx in cv_method.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Default
        model = create_pipeline()
        model.fit(X_train, y_train)
        default_scores.append(model.score(X_test, y_test))

        if show_conf_matrix:
            y_pred = model.predict(X_test)
            cm = confusion_matrix(y_test, y_pred)
            conf_matrices_default.append((cm, f"Default-Fold {fold_num}"))

        # GridSearchCV: melakukan pencarian yang lebih ekstensif dengan mencoba semua kombinasi parameter yang ditentukan dalam param_grid.
        if use_grid:
            grid = GridSearchCV(create_pipeline(), param_grid, cv=3)
            grid.fit(X_train, y_train)
            grid_scores.append(grid.best_estimator_.score(X_test, y_test))
            
            if show_conf_matrix:
                y_pred = grid.best_estimator_.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                conf_matrices_grid.append((cm, f"GridSearch-Fold {fold_num}"))

        # RandomizedSearchCV: memilih kombinasi parameter secara acak dari distribusi yang diberikan dalam param_dist, dengan tujuan mempercepat pencarian.
        if use_random:
            rand = RandomizedSearchCV(create_pipeline(), param_distributions=param_dist, n_iter=10, cv=3, random_state=42)
            rand.fit(X_train, y_train)
            random_scores.append(rand.best_estimator_.score(X_test, y_test))
            
            if show_conf_matrix:
                y_pred = rand.best_estimator_.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                conf_matrices_random.append((cm, f"RandomizedSearch-Fold {fold_num}"))

        fold_num += 1

    # Tampilkan semua confusion matrix setelah loop
    if show_conf_matrix:
        plot_multiple_confusion_matrices(conf_matrices_default, "Confusion Matrix - Default")
        if use_grid:
            plot_multiple_confusion_matrices(conf_matrices_grid, "Confusion Matrix - GridSearchCV")
        if use_random:
            plot_multiple_confusion_matrices(conf_matrices_random, "Confusion Matrix - RandomizedSearchCV")

    return default_scores, grid_scores, random_scores


# Fungsi plotting hasil
def plot_fold_results(fold_scores_dict, title):
    folds = [f'Fold {i+1}' for i in range(len(next(iter(fold_scores_dict.values()))))]
    y_pos = np.arange(len(folds))

    plt.figure(figsize=(12, 6))
    
    bar_height = 0.2
    offsets = [-bar_height, 0, bar_height]
    colors = ['gray', 'skyblue', 'lightgreen']
    labels = ['Default', 'GridSearchCV', 'RandomizedSearchCV']

    for i, (label, scores) in enumerate(fold_scores_dict.items()):
        plt.barh(y_pos + offsets[i], scores, height=bar_height, color=colors[i], label=labels[i])
        for j, score in enumerate(scores):
            plt.text(score + 0.002, y_pos[j] + offsets[i], f"{score:.4f}", va='center', ha='left')

    # Hitung mean dari masing-masing
    mean_default = np.mean(kfold_default)
    mean_grid = np.mean(kfold_grid)
    mean_random = np.mean(kfold_random)
    
    # Tambahkan titik/marker sebagai legend dan penanda mean
    plt.scatter([mean_default]*len(kfold_default), range(len(kfold_default)),
                marker='o', color='blue', label='Mean (Default)', zorder=5)
    
    plt.scatter([mean_grid]*len(kfold_grid), range(len(kfold_grid)),
                marker='x', color='navy', label='Mean (GridSearch)', zorder=5)
    
    plt.scatter([mean_random]*len(kfold_random), range(len(kfold_random)),
                marker='D', color='green', label='Mean (RandomSearch)', zorder=5)
    plt.xlim(0.75, 1.0)

    plt.xlabel('Accuracy')
    plt.ylabel('Folds')
    plt.yticks(y_pos, folds)
    plt.title(title)
    plt.legend()
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


# Evaluasi untuk KFold dan StratifiedKFold

# KFold membagi data menjadi K bagian yang tidak tumpang tindih, dan model dilatih dan diuji pada setiap bagian.
# StratifiedKFold memastikan bahwa distribusi target (positif/negatif) tetap seimbang pada setiap bagian.
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

plot_fold_results({
    'Default': kfold_default,
    'GridSearchCV': kfold_grid,
    'RandomizedSearchCV': kfold_random
}, 'KFold Cross-Validation Results')
kfold_default, kfold_grid, kfold_random = evaluate_with_cv(kfold, use_grid=True, use_random=True, show_conf_matrix=True)

plot_fold_results({
    'Default': strat_default,
    'GridSearchCV': strat_grid,
    'RandomizedSearchCV': strat_random
}, 'StratifiedKFold Cross-Validation Results')
strat_default, strat_grid, strat_random = evaluate_with_cv(strat_kfold, use_grid=True, use_random=True, show_conf_matrix=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import loguniform, randint
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Load dataset
X = X_data.copy().to_numpy()
y = y_data.copy().to_numpy()

# Pipeline builder
def create_pipeline(C=1.0):
    return Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=2)),
        ('clf', LogisticRegression(C=C, max_iter=1000, solver='liblinear'))
    ])

# Visualisasi awal PCA
def run_pipeline_with_visualization(X, y, pipeline_fn=create_pipeline, test_size=0.2, random_state=42, **pipeline_kwargs):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    pipeline = pipeline_fn(**pipeline_kwargs)
    pipeline.fit(X_train, y_train)

    X_pca_all = pipeline.named_steps['pca'].transform(
        pipeline.named_steps['scaler'].transform(X)
    )

    plt.figure(figsize=(6, 5))
    plt.title("Visualisasi PCA 2D - Sebelum Evaluasi")
    plt.scatter(X_pca_all[y == 0, 0], X_pca_all[y == 0, 1], c='blue', label='Class 0')
    plt.scatter(X_pca_all[y == 1, 0], X_pca_all[y == 1, 1], c='red', label='Class 1')
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    return pipeline

# Visualisasi PCA berdasarkan prediksi model
def plot_pca_predictions(pipeline, X, y_true, title="PCA Scatter with Predictions"):
    X_scaled = pipeline.named_steps['scaler'].transform(X)
    X_pca = pipeline.named_steps['pca'].transform(X_scaled)
    y_pred = pipeline.predict(X)

    plt.figure(figsize=(6, 5))
    plt.title(title)

    correct = y_pred == y_true
    incorrect = ~correct

    plt.scatter(X_pca[correct, 0], X_pca[correct, 1], c='green', label='Benar', alpha=0.6)
    plt.scatter(X_pca[incorrect, 0], X_pca[incorrect, 1], c='red', label='Salah', marker='x', alpha=0.6)

    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()



# Param search space
param_dist = {
    'clf__C': loguniform(0.001, 100),
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['saga'],
    'clf__max_iter': randint(2000, 5000)
}

param_grid = {
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['saga'],
    'clf__max_iter': [5000, 10000]
}

# Plot confusion matrix
def plot_multiple_confusion_matrices(conf_matrices, title):
    n = len(conf_matrices)
    fig, axes = plt.subplots(1, n, figsize=(3.5 * n, 3.5))
    if n == 1:
        axes = [axes]
    for i, (cm, name) in enumerate(conf_matrices):
        ConfusionMatrixDisplay(confusion_matrix=cm).plot(ax=axes[i], cmap='Blues', colorbar=False)
        axes[i].set_title(name)
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()
    plt.close(fig)

# Evaluasi dengan cross-validation
def evaluate_with_cv(cv_method, use_grid=False, use_random=False, show_conf_matrix=False):
    default_scores, grid_scores, random_scores = [], [], []
    conf_matrices_default, conf_matrices_grid, conf_matrices_random = [], [], []
    fold_num = 1

    for train_idx, test_idx in cv_method.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Default
        model = create_pipeline()
        model.fit(X_train, y_train)
        default_scores.append(model.score(X_test, y_test))

        if show_conf_matrix:
            y_pred = model.predict(X_test)
            cm = confusion_matrix(y_test, y_pred)
            conf_matrices_default.append((cm, f"Default-Fold {fold_num}"))

        # GridSearch
        if use_grid:
            grid = GridSearchCV(create_pipeline(), param_grid, cv=3)
            grid.fit(X_train, y_train)
            grid_scores.append(grid.best_estimator_.score(X_test, y_test))

            if show_conf_matrix:
                y_pred = grid.best_estimator_.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                conf_matrices_grid.append((cm, f"GridSearch-Fold {fold_num}"))

        # RandomizedSearch
        if use_random:
            rand = RandomizedSearchCV(create_pipeline(), param_distributions=param_dist, n_iter=10, cv=3, random_state=42)
            rand.fit(X_train, y_train)
            random_scores.append(rand.best_estimator_.score(X_test, y_test))

            if show_conf_matrix:
                y_pred = rand.best_estimator_.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                conf_matrices_random.append((cm, f"RandomSearch-Fold {fold_num}"))

        fold_num += 1

    # Plot semua confusion matrix
    if show_conf_matrix:
        plot_multiple_confusion_matrices(conf_matrices_default, "Confusion Matrix - Default")
        if use_grid:
            plot_multiple_confusion_matrices(conf_matrices_grid, "Confusion Matrix - GridSearchCV")
        if use_random:
            plot_multiple_confusion_matrices(conf_matrices_random, "Confusion Matrix - RandomizedSearchCV")

    return default_scores, grid_scores, random_scores

# Plot hasil cross-validation
def plot_fold_results(fold_scores_dict, title):
    folds = [f'Fold {i+1}' for i in range(len(next(iter(fold_scores_dict.values()))))]
    y_pos = np.arange(len(folds))
    plt.figure(figsize=(12, 6))

    bar_height = 0.2
    offsets = [-bar_height, 0, bar_height]
    colors = ['gray', 'skyblue', 'lightgreen']
    labels = list(fold_scores_dict.keys())

    for i, (label, scores) in enumerate(fold_scores_dict.items()):
        plt.barh(y_pos + offsets[i], scores, height=bar_height, color=colors[i], label=label)
        for j, score in enumerate(scores):
            plt.text(score + 0.002, y_pos[j] + offsets[i], f"{score:.4f}", va='center', ha='left')

    # Plot rata-rata
    for i, (label, scores) in enumerate(fold_scores_dict.items()):
        mean_score = np.mean(scores)
        plt.scatter([mean_score]*len(scores), y_pos + offsets[i], marker='D', s=50, label=f'Mean ({label})')
    

    plt.xlim(0.75, 1.0)
    plt.xlabel('Accuracy')
    plt.ylabel('Folds')
    plt.yticks(y_pos, folds)
    plt.title(title)
    plt.legend()
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

# Main function untuk jalankan semua langkah
def run_all(X, y):
    print(">> Visualisasi PCA")
    run_pipeline_with_visualization(X, y, pipeline_fn=create_pipeline, C=1.0)

    print(">> Evaluasi KFold")
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    kfold_default, kfold_grid, kfold_random = evaluate_with_cv(
        kfold, use_grid=True, use_random=True, show_conf_matrix=True
    )
    plot_fold_results({
        'Default': kfold_default,
        'GridSearchCV': kfold_grid,
        'RandomizedSearchCV': kfold_random
    }, 'KFold Cross-Validation Results')

    print(">> Evaluasi StratifiedKFold")
    strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    strat_default, strat_grid, strat_random = evaluate_with_cv(
        strat_kfold, use_grid=True, use_random=True, show_conf_matrix=True
    )
    plot_fold_results({
        'Default': strat_default,
        'GridSearchCV': strat_grid,
        'RandomizedSearchCV': strat_random
    }, 'StratifiedKFold Cross-Validation Results')

    print(">> Visualisasi Hasil Prediksi di PCA 2D")
    pipeline = create_pipeline(C=1.0)
    pipeline.fit(X, y)
    plot_pca_predictions(pipeline, X, y, title="PCA Scatter Based on Predictions")


# Jalankan semuanya
run_all(X, y)


#### K-Nearest Neighbor
Alasan Utama: Model berbasis kedekatan yang sangat fleksibel untuk data non-linier.

Penjelasan: KNN bekerja dengan melihat data yang "terdekat" dengan contoh yang ingin diprediksi. Dalam konteks data medis, ini sangat berguna, karena pola dalam data mungkin tidak linier. Misalnya, dua pasien dengan tekanan darah yang sama mungkin memiliki hasil berbeda tergantung pada faktor-faktor lain seperti usia atau detak jantung.

Kenapa Memilih Ini: KNN sangat cocok ketika hubungan antara fitur dan target tidak bisa dijelaskan secara linier. Jika model Logistic Regression gagal menangkap pola yang kompleks dalam data, KNN bisa memberikan alternatif yang lebih fleksibel dengan menggunakan kedekatan antar data untuk memprediksi hasil. Ini juga memberikan hasil yang baik tanpa perlu banyak tuning.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint, uniform

X = X_data.copy().to_numpy()
y = y_data.copy().to_numpy()

# Function untuk membuat pipeline model: Scaling sangat penting pada KNN karena berbasis jarak.
def create_pipeline(n_neighbors=5, weights='uniform', metric='minkowski'):
    return Pipeline([
        ('scaler', StandardScaler()),
        ('clf', KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric))
    ])
# weights: apakah semua tetangga bobotnya sama (uniform) atau dihitung berdasarkan jarak (distance).
# metric: jenis metrik jarak, misalnya Euclidean atau Manhattan.

# Parameter untuk RandomizedSearchCV
param_dist = {
    'clf__n_neighbors': randint(1, 20),
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['minkowski', 'euclidean', 'manhattan'],
}

# Parameter untuk GridSearchCV
param_grid = {
    'clf__n_neighbors': [3, 5, 7, 9, 11],
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['minkowski', 'euclidean', 'manhattan'],
}

def plot_multiple_confusion_matrices(conf_matrices, title):
    n = len(conf_matrices)
    fig, axes = plt.subplots(1, n, figsize=(3.5 * n, 3.5))

    if n == 1:
        axes = [axes]

    for i, (cm, name) in enumerate(conf_matrices):
        ConfusionMatrixDisplay(confusion_matrix=cm).plot(ax=axes[i], cmap='Blues', colorbar=False)
        axes[i].set_title(name)

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()  # Pastikan hanya panggil show sekali di sini
    plt.close(fig)  # ⬅⬅⬅ Tambahkan ini untuk menghindari dobel


# Fungsi evaluasi untuk KFold dan StratifiedKFold
def evaluate_with_cv(cv_method, use_grid=False, use_random=False, show_conf_matrix=False):
    default_scores = []
    grid_scores = []
    random_scores = []
    fold_num = 1

    # Untuk menyimpan semua confusion matrices
    conf_matrices_default = []
    conf_matrices_grid = []
    conf_matrices_random = []

    for train_idx, test_idx in cv_method.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Default
        model = create_pipeline()
        model.fit(X_train, y_train)
        default_scores.append(model.score(X_test, y_test))

        if show_conf_matrix:
            y_pred = model.predict(X_test)
            cm = confusion_matrix(y_test, y_pred)
            conf_matrices_default.append((cm, f"Default-Fold {fold_num}"))

        # GridSearchCV
        if use_grid:
            grid = GridSearchCV(create_pipeline(), param_grid, cv=3)
            grid.fit(X_train, y_train)
            grid_scores.append(grid.best_estimator_.score(X_test, y_test))
            
            if show_conf_matrix:
                y_pred = grid.best_estimator_.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                conf_matrices_grid.append((cm, f"GridSearch-Fold {fold_num}"))

        # RandomizedSearchCV
        if use_random:
            rand = RandomizedSearchCV(create_pipeline(), param_distributions=param_dist, n_iter=10, cv=3, random_state=42)
            rand.fit(X_train, y_train)
            random_scores.append(rand.best_estimator_.score(X_test, y_test))
            
            if show_conf_matrix:
                y_pred = rand.best_estimator_.predict(X_test)
                cm = confusion_matrix(y_test, y_pred)
                conf_matrices_random.append((cm, f"RandomizedSearch-Fold {fold_num}"))

        fold_num += 1

    # Tampilkan semua confusion matrix setelah loop
    if show_conf_matrix:
        plot_multiple_confusion_matrices(conf_matrices_default, "Confusion Matrix - Default")
        if use_grid:
            plot_multiple_confusion_matrices(conf_matrices_grid, "Confusion Matrix - GridSearchCV")
        if use_random:
            plot_multiple_confusion_matrices(conf_matrices_random, "Confusion Matrix - RandomizedSearchCV")

    return default_scores, grid_scores, random_scores

# Fungsi plotting hasil
def plot_fold_results(fold_scores_dict, title, labels):
    folds = [f'Fold {i+1}' for i in range(len(next(iter(fold_scores_dict.values()))))]
    y_pos = np.arange(len(folds))

    plt.figure(figsize=(12, 6))

    bar_height = 0.2
    offsets = [-bar_height, 0, bar_height]
    colors = ['gray', 'skyblue', 'lightgreen']
    labels_legend = ['Default', 'GridSearchCV', 'RandomizedSearchCV']

    for i, (label, scores) in enumerate(fold_scores_dict.items()):
        plt.barh(y_pos + offsets[i], scores, height=bar_height, color=colors[i], label=labels_legend[i])
        for j, score in enumerate(scores):
            plt.text(score + 0.002, y_pos[j] + offsets[i], f"{score:.4f}", va='center', ha='left')

    # Hitung mean dari masing-masing
    mean_default = np.mean(kfold_default)
    mean_grid = np.mean(kfold_grid)
    mean_random = np.mean(kfold_random)

    # Tambahkan titik/marker sebagai legend dan penanda mean
    plt.scatter([mean_default]*len(kfold_default), range(len(kfold_default)),
                marker='o', color='blue', label='Mean (Default)', zorder=5)

    plt.scatter([mean_grid]*len(kfold_grid), range(len(kfold_grid)),
                marker='x', color='navy', label='Mean (GridSearch)', zorder=5)

    plt.scatter([mean_random]*len(kfold_random), range(len(kfold_random)),
                marker='D', color='green', label='Mean (RandomSearch)', zorder=5)
    plt.xlim(0.75, 1.0)

    plt.xlabel('Accuracy')
    plt.ylabel('Folds')
    plt.yticks(y_pos, folds)
    plt.title(title)
    plt.legend()
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


# Evaluasi untuk KFold dan StratifiedKFold
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

plot_fold_results({
    'Default': kfold_default,
    'GridSearchCV': kfold_grid,
    'RandomizedSearchCV': kfold_random
}, 'KFold Cross-Validation Results', ['Default', 'GridSearchCV', 'RandomizedSearchCV'])
kfold_default, kfold_grid, kfold_random = evaluate_with_cv(kfold, use_grid=True, use_random=True, show_conf_matrix=True)

plot_fold_results({
    'Default': strat_default,
    'GridSearchCV': strat_grid,
    'RandomizedSearchCV': strat_random
}, 'StratifiedKFold Cross-Validation Results', ['Default', 'GridSearchCV', 'RandomizedSearchCV'])
strat_default, strat_grid, strat_random = evaluate_with_cv(strat_kfold, use_grid=True, use_random=True, show_conf_matrix=True)

### Ensemble Learning memakai XGBoost dan Adaboost
Alasan Utama: Menggabungkan kekuatan beberapa model untuk hasil yang lebih kuat dan stabil.

Penjelasan: Model voting menggabungkan hasil dari berbagai model yang berbeda untuk menghasilkan prediksi akhir. Dalam kasus ini, kita menggabungkan:

- XGBoost: Model boosting yang sangat kuat, dapat menangani data besar dan kompleks, serta cenderung memberikan hasil yang sangat baik dalam waktu yang efisien.

- AdaBoost: Fokus pada memperbaiki kelemahan model sebelumnya dengan memberi bobot lebih pada data yang salah diklasifikasikan, meningkatkan akurasi secara bertahap.

- KNN: Menambahkan pendekatan berbasis kedekatan yang memperkaya model dengan kemampuan mendeteksi pola lokal yang mungkin terlewatkan oleh model lain.

Kenapa Memilih Ini: Dengan menggabungkan beberapa model berbeda, kita bisa memanfaatkan kelebihan masing-masing: XGBoost dan AdaBoost memberikan kekuatan dalam menangani kesalahan prediksi dan pola yang lebih kompleks, sementara KNN memperkenalkan elemen berbasis kedekatan. Ini memungkinkan model untuk bekerja lebih baik di berbagai kondisi dan mengurangi overfitting atau underfitting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint, uniform

# Load dataset
X = X_data.copy().to_numpy()
y = y_data.copy().to_numpy()

# Define parameter grids for GridSearchCV and RandomizedSearchCV
param_xgb = {
    # 'xgb__n_estimators': [50, 100, 150, 200],
    # 'xgb__max_depth': [3, 5, 7, 10],
    'xgb__learning_rate': [0.01, 0.1, 0.2, 0.3]
}
param_ada = {
    # 'ada__n_estimators': [50, 100, 150, 200],
    'ada__learning_rate': [0.01, 0.1, 0.5, 1.0]
}
param_xgb_random = {
    # 'xgb__n_estimators': randint(50, 200),
    # 'xgb__max_depth': randint(3, 10),
    'xgb__learning_rate': uniform(0.01, 0.3)
}
param_ada_random = {
    # 'ada__n_estimators': randint(50, 200),
    'ada__learning_rate': uniform(0.01, 1.0)
}

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def test_fold(cv_method, search_method, param_xgb, param_ada):
    default_score = []
    tuned_score = []
    cm_default = []  # Menyimpan confusion matrix untuk default
    cm_tuned = []    # Menyimpan confusion matrix untuk model tuning

    for train_index, test_index in cv_method.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Default models
        xgb = XGBClassifier()
        ada = AdaBoostClassifier(algorithm='SAMME')
        knn = KNeighborsClassifier()

        # Default ensemble
        ensemble_default = Pipeline([
            ('scaler', StandardScaler()),
            ('vote', VotingClassifier(estimators=[
                ('xgb', xgb),
                ('ada', ada),
                ('knn', knn)
            ], voting='soft'))
        ])
        ensemble_default.fit(X_train, y_train)
        default_score.append(ensemble_default.score(X_test, y_test))

        # Calculate confusion matrix for default
        cm_default.append(confusion_matrix(y_test, ensemble_default.predict(X_test)))

        # Hyperparameter tuning
        xgb_pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('xgb', XGBClassifier())
        ])
        if search_method == 'grid':
            xgb_search = GridSearchCV(xgb_pipe, param_grid=param_xgb, cv=3)
        else:
            xgb_search = RandomizedSearchCV(xgb_pipe, param_distributions=param_xgb_random, n_iter=10, cv=3, random_state=42)
        
        xgb_search.fit(X_train, y_train)

        ada_pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('ada', AdaBoostClassifier(algorithm='SAMME'))
        ])
        if search_method == 'grid':
            ada_search = GridSearchCV(ada_pipe, param_grid=param_ada, cv=3)
        else:
            ada_search = RandomizedSearchCV(ada_pipe, param_distributions=param_ada_random, n_iter=10, cv=3, random_state=42)
        
        ada_search.fit(X_train, y_train)

        # Tuned ensemble
        ensemble_tuned = Pipeline([
            ('scaler', StandardScaler()),
            ('vote', VotingClassifier(estimators=[
                ('xgb', xgb_search.best_estimator_.named_steps['xgb']),
                ('ada', ada_search.best_estimator_.named_steps['ada']),
                ('knn', knn)
            ], voting='soft'))
        ])
        ensemble_tuned.fit(X_train, y_train)
        tuned_score.append(ensemble_tuned.score(X_test, y_test))

        # Calculate confusion matrix for tuned
        cm_tuned.append(confusion_matrix(y_test, ensemble_tuned.predict(X_test)))

    return default_score, tuned_score, cm_default, cm_tuned

def plot_comparison(score_default, score_grid, score_random, cv_type, cm_default, cm_grid, cm_random):
    folds = [f'Fold {i+1}' for i in range(len(score_default))]
    x = np.arange(len(folds))
    bar_width = 0.25

    plt.figure(figsize=(12, 6))

    # Plot for Default (no tuning)
    plt.barh(x - bar_width, score_default, height=bar_width, label='Default (No Tuning)', color='gray')

    # Plot for GridSearchCV
    plt.barh(x, score_grid, height=bar_width, label='GridSearchCV Tuning', color='skyblue')

    # Plot for RandomizedSearchCV
    plt.barh(x + bar_width, score_random, height=bar_width, label='RandomizedSearchCV Tuning', color='lightgreen')

    for i in range(len(folds)):
        plt.text(score_default[i] + 0.001, x[i] - bar_width, f"{score_default[i]:.4f}", va='center')
        plt.text(score_grid[i] + 0.001, x[i], f"{score_grid[i]:.4f}", va='center')
        plt.text(score_random[i] + 0.001, x[i] + bar_width, f"{score_random[i]:.4f}", va='center')

    # Hitung mean dari masing-masing
    mean_default = np.mean(score_default)
    mean_grid = np.mean(score_grid)
    mean_random = np.mean(score_random)

    # Tambahkan titik/marker sebagai legend dan penanda mean
    plt.scatter([mean_default]*len(score_default), range(len(score_default)),
                marker='o', color='blue', label='Mean (Default)', zorder=5)

    plt.scatter([mean_grid]*len(score_grid), range(len(score_grid)),
                marker='x', color='navy', label='Mean (GridSearch)', zorder=5)

    plt.scatter([mean_random]*len(score_random), range(len(score_random)),
                marker='D', color='green', label='Mean (RandomSearch)', zorder=5)
    
    plt.xlim(0.75, 1.0)
    plt.yticks(x, folds)
    plt.xlabel('Accuracy')
    plt.title(f'Comparison of Voting Classifier Accuracies ({cv_type})')
    plt.legend()
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Menampilkan Confusion Matrix untuk setiap metode
    for method, cm in zip(['Default', 'GridSearchCV', 'RandomizedSearchCV'], [cm_default, cm_grid, cm_random]):
        fig, axes = plt.subplots(1, 5, figsize=(20, 4))  # 1 baris, 5 kolom
        fig.suptitle(f'Confusion Matrices - {method}', fontsize=16)

        for i, cm_fold in enumerate(cm):
            disp = ConfusionMatrixDisplay(confusion_matrix=cm_fold)
            disp.plot(cmap='Blues', values_format='d', ax=axes[i], colorbar=False)
            axes[i].set_title(f'Fold {i+1}')
        
        plt.tight_layout(rect=[0, 0, 1, 0.95])  # Supaya suptitle tidak kepotong
        plt.show()

# KFold + Default
kf = KFold(n_splits=5, shuffle=True, random_state=42)
score_default_kf, score_grid_kf, score_random_kf = [], [], []
cm_default_kf, cm_grid_kf, cm_random_kf = [], [], []
for method in ['none', 'grid', 'random']:
    score_default, score_tuned, cm_default, cm_tuned = test_fold(kf, method, param_xgb, param_ada)
    if method == 'none':
        score_default_kf = score_default
        cm_default_kf = cm_default
    elif method == 'grid':
        score_grid_kf = score_tuned
        cm_grid_kf = cm_tuned
    else:
        score_random_kf = score_tuned
        cm_random_kf = cm_tuned

# StratifiedKFold + Default
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
score_default_skf, score_grid_skf, score_random_skf = [], [], []
cm_default_skf, cm_grid_skf, cm_random_skf = [], [], []
for method in ['none', 'grid', 'random']:
    score_default, score_tuned, cm_default, cm_tuned = test_fold(skf, method, param_xgb, param_ada)
    if method == 'none':
        score_default_skf = score_default
        cm_default_skf = cm_default
    elif method == 'grid':
        score_grid_skf = score_tuned
        cm_grid_skf = cm_tuned
    else:
        score_random_skf = score_tuned
        cm_random_skf = cm_tuned

# Plot results for KFold
plot_comparison(score_default_kf, score_grid_kf, score_random_kf, "K-Fold", cm_default_kf, cm_grid_kf, cm_random_kf)

# Plot results for StratifiedKFold
plot_comparison(score_default_skf, score_grid_skf, score_random_skf, "Stratified K-Fold", cm_default_skf, cm_grid_skf, cm_random_skf)

#### Kesimpulan Visualisasi:

Secara umum, Stratified K-Fold memberikan hasil yang sedikit lebih stabil dan konsisten antar fold dibandingkan dengan K-Fold biasa. Hal ini terlihat dari sebaran nilai akurasi yang cenderung lebih merata pada Stratified K-Fold, sedangkan pada K-Fold biasa terdapat variasi yang cukup signifikan antar fold (contoh: Fold 4 di K-Fold sering drop akurasinya secara drastis). Stratified K-Fold menjaga proporsi kelas target pada setiap fold, sementara K-Fold biasa tidak, sehingga rawan terjadi pembagian data yang tidak seimbang, apalagi pada dataset yang tidak seimbang (imbalanced class).

Kesimpulan antar fold menunjukkan bahwa Fold 2 dan Fold 1 sering memberikan akurasi lebih tinggi dan stabil, terutama pada Stratified K-Fold. Sementara itu, Fold 4 dan 5 cenderung jadi fold yang paling menantang (dengan performa turun) pada K-Fold biasa. Berdasarkan keseluruhan grafik, metode dengan Stratified K-Fold + GridSearchCV atau RandomizedSearchCV umumnya memberikan kombinasi akurasi tertinggi dan paling stabil, sehingga dapat disimpulkan sebagai metode yang paling optimal.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import randint, uniform

# Load dataset
X = X_data.copy().to_numpy()
y = y_data.copy().to_numpy()

# Define parameter grids for GridSearchCV and RandomizedSearchCV
param_xgb = {
    'xgb__n_estimators': [50, 100, 150, 200],
    'xgb__max_depth': [3, 5, 7, 10],
    'xgb__learning_rate': [0.01, 0.1, 0.2, 0.3]
}
param_ada = {
    'ada__n_estimators': [50, 100, 150, 200],
    'ada__learning_rate': [0.01, 0.1, 0.5, 1.0]
}
param_xgb_random = {
    'xgb__n_estimators': randint(50, 200),
    'xgb__max_depth': randint(3, 10),
    'xgb__learning_rate': uniform(0.01, 0.3)
}
param_ada_random = {
    'ada__n_estimators': randint(50, 200),
    'ada__learning_rate': uniform(0.01, 1.0)
}

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def test_fold(cv_method, search_method, param_xgb, param_ada):
    default_score = []
    tuned_score = []
    cm_default = []  # Menyimpan confusion matrix untuk default
    cm_tuned = []    # Menyimpan confusion matrix untuk model tuning

    for train_index, test_index in cv_method.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Default models
        xgb = XGBClassifier()
        ada = AdaBoostClassifier(algorithm='SAMME')
        knn = KNeighborsClassifier()

        # Default ensemble
        ensemble_default = Pipeline([
            ('scaler', StandardScaler()),
            ('vote', VotingClassifier(estimators=[
                ('xgb', xgb),
                ('ada', ada),
                ('knn', knn)
            ], voting='soft'))
        ])
        ensemble_default.fit(X_train, y_train)
        default_score.append(ensemble_default.score(X_test, y_test))

        # Calculate confusion matrix for default
        cm_default.append(confusion_matrix(y_test, ensemble_default.predict(X_test)))

        # Hyperparameter tuning
        xgb_pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('xgb', XGBClassifier())
        ])
        if search_method == 'grid':
            xgb_search = GridSearchCV(xgb_pipe, param_grid=param_xgb, cv=3)
        else:
            xgb_search = RandomizedSearchCV(xgb_pipe, param_distributions=param_xgb_random, n_iter=10, cv=3, random_state=42)
        
        xgb_search.fit(X_train, y_train)

        ada_pipe = Pipeline([
            ('scaler', StandardScaler()),
            ('ada', AdaBoostClassifier(algorithm='SAMME'))
        ])
        if search_method == 'grid':
            ada_search = GridSearchCV(ada_pipe, param_grid=param_ada, cv=3)
        else:
            ada_search = RandomizedSearchCV(ada_pipe, param_distributions=param_ada_random, n_iter=10, cv=3, random_state=42)
        
        ada_search.fit(X_train, y_train)

        # Tuned ensemble
        ensemble_tuned = Pipeline([
            ('scaler', StandardScaler()),
            ('vote', VotingClassifier(estimators=[
                ('xgb', xgb_search.best_estimator_.named_steps['xgb']),
                ('ada', ada_search.best_estimator_.named_steps['ada']),
                ('knn', knn)
            ], voting='soft'))
        ])
        ensemble_tuned.fit(X_train, y_train)
        tuned_score.append(ensemble_tuned.score(X_test, y_test))

        # Calculate confusion matrix for tuned
        cm_tuned.append(confusion_matrix(y_test, ensemble_tuned.predict(X_test)))

    return default_score, tuned_score, cm_default, cm_tuned

def plot_comparison(score_default, score_grid, score_random, cv_type, cm_default, cm_grid, cm_random):
    folds = [f'Fold {i+1}' for i in range(len(score_default))]
    x = np.arange(len(folds))
    bar_width = 0.25

    plt.figure(figsize=(12, 6))

    # Plot for Default (no tuning)
    plt.barh(x - bar_width, score_default, height=bar_width, label='Default (No Tuning)', color='gray')

    # Plot for GridSearchCV
    plt.barh(x, score_grid, height=bar_width, label='GridSearchCV Tuning', color='skyblue')

    # Plot for RandomizedSearchCV
    plt.barh(x + bar_width, score_random, height=bar_width, label='RandomizedSearchCV Tuning', color='lightgreen')

    for i in range(len(folds)):
        plt.text(score_default[i] + 0.001, x[i] - bar_width, f"{score_default[i]:.4f}", va='center')
        plt.text(score_grid[i] + 0.001, x[i], f"{score_grid[i]:.4f}", va='center')
        plt.text(score_random[i] + 0.001, x[i] + bar_width, f"{score_random[i]:.4f}", va='center')

    # Hitung mean dari masing-masing
    mean_default = np.mean(score_default)
    mean_grid = np.mean(score_grid)
    mean_random = np.mean(score_random)

    # Tambahkan titik/marker sebagai legend dan penanda mean
    plt.scatter([mean_default]*len(score_default), range(len(score_default)),
                marker='o', color='blue', label='Mean (Default)', zorder=5)

    plt.scatter([mean_grid]*len(score_grid), range(len(score_grid)),
                marker='x', color='navy', label='Mean (GridSearch)', zorder=5)

    plt.scatter([mean_random]*len(score_random), range(len(score_random)),
                marker='D', color='green', label='Mean (RandomSearch)', zorder=5)
    
    plt.xlim(0.75, 1.0)
    plt.yticks(x, folds)
    plt.xlabel('Accuracy')
    plt.title(f'Comparison of Voting Classifier Accuracies ({cv_type})')
    plt.legend()
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Menampilkan Confusion Matrix untuk setiap metode
    for method, cm in zip(['Default', 'GridSearchCV', 'RandomizedSearchCV'], [cm_default, cm_grid, cm_random]):
        fig, axes = plt.subplots(1, 5, figsize=(20, 4))  # 1 baris, 5 kolom
        fig.suptitle(f'Confusion Matrices - {method}', fontsize=16)

        for i, cm_fold in enumerate(cm):
            disp = ConfusionMatrixDisplay(confusion_matrix=cm_fold)
            disp.plot(cmap='Blues', values_format='d', ax=axes[i], colorbar=False)
            axes[i].set_title(f'Fold {i+1}')
        
        plt.tight_layout(rect=[0, 0, 1, 0.95])  # Supaya suptitle tidak kepotong
        plt.show()

# KFold + Default
kf = KFold(n_splits=5, shuffle=True, random_state=42)
score_default_kf, score_grid_kf, score_random_kf = [], [], []
cm_default_kf, cm_grid_kf, cm_random_kf = [], [], []
for method in ['none', 'grid', 'random']:
    score_default, score_tuned, cm_default, cm_tuned = test_fold(kf, method, param_xgb, param_ada)
    if method == 'none':
        score_default_kf = score_default
        cm_default_kf = cm_default
    elif method == 'grid':
        score_grid_kf = score_tuned
        cm_grid_kf = cm_tuned
    else:
        score_random_kf = score_tuned
        cm_random_kf = cm_tuned

# StratifiedKFold + Default
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
score_default_skf, score_grid_skf, score_random_skf = [], [], []
cm_default_skf, cm_grid_skf, cm_random_skf = [], [], []
for method in ['none', 'grid', 'random']:
    score_default, score_tuned, cm_default, cm_tuned = test_fold(skf, method, param_xgb, param_ada)
    if method == 'none':
        score_default_skf = score_default
        cm_default_skf = cm_default
    elif method == 'grid':
        score_grid_skf = score_tuned
        cm_grid_skf = cm_tuned
    else:
        score_random_skf = score_tuned
        cm_random_skf = cm_tuned

# Plot results for KFold
plot_comparison(score_default_kf, score_grid_kf, score_random_kf, "K-Fold", cm_default_kf, cm_grid_kf, cm_random_kf)

# Plot results for StratifiedKFold
plot_comparison(score_default_skf, score_grid_skf, score_random_skf, "Stratified K-Fold", cm_default_skf, cm_grid_skf, cm_random_skf)